# Tanzania Primary Education Results (NECTA PSLE)

### 2a. Data Sourcing - TAMISEMI

**Comment:** *Very thankful that this data is publically accessible!*

* Light data cleaning on dataset before every merge
1. Merge with TAMISEMI 1: consolidated school data (Gov/Non, location, PTR)
2. Merge with TAMISEMI 2: primary textbooks school data (PBR)
3. Merge with TAMISEMI 3: capitation grant school data (CG) 
4. Merge with TAMISEMI 4: enrollment by age and sex school data

#### Inputs:
* **nation_necta_features.csv (17900, 12)**

#### Outputs:
* **nation_merged.csv (17900, 28)**
* df_ts11_fixes.csv (35, 19)
* df_x12_left_fixes.csv (80, 12)
* df_x12_left.csv (2, 12)
* df_x12_both.csv (567, 27)
* df_x13_duplicated_first.csv (24, 27)
    
#### Functions:
* `calc_pbr_average`
* `get_ages_stats`

In [1]:
#Libraries: pre-installed in Anaconda
import numpy as np
import pandas as pd
#User-defined functions.py
import functions as fn

### 1. Merge with TAMISEMI 1: consolidated school data (Gov/Non, location, PTR)
### A. Data Cleaning
**ELI5 Summary:**
*Check main school features data for obvious issues and corner cases*

**Steps:**
* Light data cleaning: dtypes, values, duplicates, drop columns, missing data
* Light feature extraction (and data clean)

**DATA observations:**
* This is the MAIN dataset from [per-school data](https://www.tamisemi.go.tz/singleministers/basic-education-data-2022) from TAMISEMI (President's Office - Regional Administration and Local Government) including interesting features:
    * **Ownership:** Government/Non-Government
    * **Location:** administrative down to ward (neighborhood) and latitude/longitude coordinates
    * **Pupil-to-Teacher Ratio (PTR)**
* Full dataset is **19,261** primary schools registered in 2022
    * After data cleaning, only **17,611** have a NECTA ID# from 2021

**Corner cases:**
* **16 duplicated** `'NECTA EXAM CENTRE NO'`
    * 7 corrected from NECTA web
    * 9 corrected to n/a (not found on NECTA web)
* 3 corrected (1 from n/a) `'NECTA EXAM CENTRE NO'` from Morogoro Region detailed analysis
* 5 schools removed with zero `'TOTAL STUDENTS'`
* 6 schools with infinite `'PQTR'` (zero qualified teachers) => use `np.inf`

**Learnings:** (🧑🏻‍💻📚😎⚠️)
- 🧑🏻‍💻 Handy sanity check code first used here
    - MISSING values: `.isna().sum()`
    - Duplicate rows: `.duplicated(keep=False).sum()` (then divide by two)
- 🧑🏻‍💻 NumPy `np.isinf()` finds `inf` (infinite) values from division by 0
- 📚 Data cleaning ***as immediately as possible*** pays off here for cleaned, unique, understood data BEFORE the merges
- ⚠️ Python round() for n.5 rounds to nearest **even** number!

In [2]:
#Prepare TAMISEMI dataset #1 "Consolidated" for joining
df_ts1 = pd.read_csv('../datain/Consolidated_Primary_EnrolmentbyGrade_PTR_2022_PSLE2021.csv',\
                     usecols = ['REGION', 'COUNCIL', 'WARD', 'SCHOOL NAME', 'SCHOOL REG. NUMBER', 'NECTA EXAM CENTRE NO',\
                                'SCHOOL OWNERSHIP', 'LATITUTE', 'LONGITUDE',\
                                'TOTAL BOYS', 'TOTAL GIRLS',\
                                'ALL TEACHERS MALE', 'ALL TEACHERS FEMALE', 'QUALIFIED TEACHERS MALE', 'QUALIFIED TEACHERS FEMALE',\
                                'PTR', 'PQTR'])
df_ts1.shape

(19261, 17)

In [16]:
# Light Data Cleaning

#(1) Data types
df_ts11 = df_ts1.convert_dtypes() #returns a copy
categorical_list = ['REGION', 'COUNCIL', 'SCHOOL OWNERSHIP']
df_ts11[categorical_list] = df_ts11[categorical_list].astype('category')

#(2) Data values

#Rename column
df_ts11.rename(columns={'LATITUTE': 'LATITUDE'}, inplace=True)
#Remove '-' to match NECTA
df_ts11['NECTA EXAM CENTRE NO'] = df_ts11['NECTA EXAM CENTRE NO'].apply(lambda x : "".join(str(x).split('-')))
df_ts11['NECTA EXAM CENTRE NO'] = df_ts11['NECTA EXAM CENTRE NO'].replace('<NA>', pd.NA) #undo from previous apply

#Replace PQTR 'divide by 0s' when sum(QUALIFIED TEACHERS) = 0
df_ts11['PQTR'] = df_ts11['PQTR'].astype('object').replace('#DIV/0!', np.inf).astype('float64')
#df_ts11[np.isinf(df_ts11['PQTR'])] #result: 6 rows (sum(QUALIFIED) = 0)

#Data integrity check PTR/PQTR
df_ts11['TOTAL STUDENTS'] = df_ts11['TOTAL GIRLS'] + df_ts11['TOTAL BOYS']
df_ts11['PTR CHECK'] = (df_ts11['TOTAL STUDENTS'] / (df_ts11['ALL TEACHERS FEMALE'] + df_ts11['ALL TEACHERS MALE']) + 0.5).astype(int)
assert (df_ts11['PTR CHECK'] == df_ts11['PTR']).all(), 'Found mismatch in PTR calculation'

df_ts11notinf = df_ts11[np.isfinite(df_ts11['PQTR'])].copy() #don't check np.inf cases
df_ts11notinf['PQTR CHECK'] = (df_ts11notinf['TOTAL STUDENTS'] / (df_ts11notinf['QUALIFIED TEACHERS FEMALE'] + df_ts11notinf['QUALIFIED TEACHERS MALE']) + 0.5).astype(int)
assert (df_ts11notinf['PQTR CHECK'] == df_ts11notinf['PQTR'].astype('int')).all(), 'Found mismatch in PQTR calculation'

In [7]:
#(3) Duplicate rows - NECTA ID#
#check
#df_ts11[df_ts11['NECTA EXAM CENTRE NO'].notna()].duplicated(subset=['NECTA EXAM CENTRE NO']).sum()

#Get duplicated NECTA ID#
df_ts11notna = df_ts11[df_ts11['NECTA EXAM CENTRE NO'].notna()] #(17619, 17)
df_ts11dup = df_ts11notna[df_ts11notna['NECTA EXAM CENTRE NO'].duplicated(keep=False)] #(32, 17)

#Found mixed up NECTA ID# among three schools in Morogoro Region
df_ts11err = df_ts11[(df_ts11['SCHOOL NAME'].apply(lambda x : x.startswith('CHITA'))) & (df_ts11['WARD'] == 'Chita')]

#output to CSV for MANUAL Excel corrections
pd.concat([df_ts11dup, df_ts11err]).to_csv('dataout/2a/df_ts11_duplicates_errors.csv')

################################
### MANUAL Excel corrections ###
################################

#read from CSV after MANUAL Excel corrections
df_ts11fixes = pd.read_csv('dataout/2a/df_ts11_fixes.csv', index_col=0).convert_dtypes()
#concat to original TS1, then drop_duplicates keeping the 'last' FIXED entry
df_ts12 = pd.concat([df_ts11, df_ts11fixes])
df_ts13 = df_ts12.drop_duplicates(subset=['SCHOOL REG. NUMBER'], keep='last').copy()

#check
df_ts13[df_ts13['NECTA EXAM CENTRE NO'].notna()].duplicated(subset=['NECTA EXAM CENTRE NO']).sum()

0

In [8]:
#(4) Drop unneeded columns
df_ts14 = df_ts13.drop(['ALL TEACHERS MALE', 'ALL TEACHERS FEMALE', 'QUALIFIED TEACHERS MALE', 'QUALIFIED TEACHERS FEMALE', 'PTR CHECK'], axis=1)
#df_ts14.info()

#(5) Check MISSING data (keep for merge)
#df_ts14.isna().any(axis=0) #NECTA EXAM CENTRE NO     True
df_ts14.isna().any(axis=1).sum()

#Light Feature Extraction (and data clean)
df_ts15 = df_ts14[df_ts14['TOTAL STUDENTS'] != 0].copy() #drop now to avoid 0/0 for RATIO
df_ts15['RATIO GIRLS-BOYS'] = df_ts15['TOTAL GIRLS'] / df_ts15['TOTAL BOYS']
#df_ts15[np.isinf(df_ts15['RATIO GIRLS-BOYS'])] #girls-only schools

In [9]:
#Save to intermediate CSV
#df_ts15.to_csv('dataout/2a/df_ts15_cleaned.csv')
df_ts15.shape

(19256, 15)

### 1. Merge with TAMISEMI 1: consolidated school data (Gov/Non, location, PTR)
### B. Merge NECTA with TAMISEMI 1
**ELI5 Summary:**
*Combine examination results data with main school features data using SQL-like JOINS [(Pandas merge)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)*

**Steps:**
- Patch in council (4) and school (80) name mismatch FIXES discovered in 1.2 LEFT JOIN (left_only)
1. LEFT JOIN **on school exam ID** (both = **17,331** schools) < read in CSV
2. LEFT JOIN **on school name and council** (both = **567** schools) < left_only from step 1
3. Concatenate joined (and **2 unjoined**) schools (total = **17,900** schools) < both from step 1, both and left_only from step 2
4. Post-processing steps: internal checks, drop unneeded columns, set index to NECTA exam ID > save CSV

**DATA observations:**
* Full NECTA dataset is **17,900** primary schools (clean results)
    * 7.1% of registered schools didn't do exams

**Corner cases:**
* Discovered during JOINs:
    * (1.1) Found **569 schools** (left_only) with mismatched or no (new in 2022?) exam ID in  TAMISEMI dataset => mostly from 3 councils
    * (1.2) Found **80 schools** without exact council and school name match
        * (1.2) Fixed **78 mismatched** school names
        * (1.2) Remaining **2 schools** (left_only) not identifable in TAMISEMI set **=> KEEP (NECTA data only)**
    * (1.4) Found **24 "incorrectly" matched schools** from 1.1 **=> unmatch and KEEP (NECTA data only)**

**Learnings:** (🧑🏻‍💻📚😎⚠️)
- 🧑🏻‍💻 Use `indicator=True` to get **indicator column**: `both`, `left_only` for sane construction of full dataset
- 🧑🏻‍💻 Name DataFrames by flow sequence, e.g., df_x13 (assignment to new DF at most steps)
- 😎 A meaningful index can be helpful!, e.g., `'SCHOOL REG. NUMBER'` here for subsequent TAMISEMI dataset merges
- ⚠️ Small string mismatches (case, punctuation, spelling, whitespace) require manual checks
- ⚠️ **There can be corner cases of corner cases!** *Just when you think you're done* (see 1.2 above)

In [10]:
#Read in CSV
n_dtype_dict = {'school_name': 'string', 'school_id': 'string',
                'grade': 'category', 'region_name': 'category', 'council_name': 'category'}
df_n = pd.read_csv('dataout/2a/nation_necta_features.csv', index_col=0, dtype=n_dtype_dict)
df_n.shape

(17900, 12)

In [11]:
#Patch in FIXES discovered in 1.2 LEFT JOIN (below)
df_n2 = df_n.copy()
df_ts16 = df_ts15.copy()

#1.01 - FIX 'COUNCIL' mismatches, replace with NECTA council names
to_replace_list = ['Kahama MC', 'Kigoma/Ujiji MC', 'Mtwara Mikindani MC']
value_list = ['Kahama TC', 'Kigoma MC', 'Mtwara MC']
df_ts16['COUNCIL'] = df_ts16['COUNCIL'].replace(to_replace_list, value_list)

#1.02 - Capital "S" in Dar es Salaam
df_n2['council_name'] = df_n2['council_name'].replace('Dar es salaam CC', 'Dar es Salaam CC')
df_n2['region_name'] = df_n2['region_name'].replace('Dar es salaam', 'Dar es Salaam')

#1.03 - FIX 'school_name' mismatches, replace with TAMISEMI school names
df_n_fixes = pd.read_csv('dataout/2a/df_x12_left_fixes.csv', index_col=0, dtype=n_dtype_dict)

#concat to original TS1, then drop_duplicates keeping the 'last' FIXED entry
df_n3 = pd.concat([df_n2, df_n_fixes])
df_n4 = df_n3.drop_duplicates(subset=['school_id'], keep='last')

#check
df_n4.shape

(17900, 12)

In [12]:
#1.1 LEFT JOIN on school exam ID (most cases)
df_x11 = df_n4.merge(df_ts16, how='left', indicator=True,\
                     left_on=['school_id'],\
                     right_on=['NECTA EXAM CENTRE NO'])

df_x11.shape
#df_x11.to_csv('dataout/2a/df_x11_merged.csv')

#Split joined and unjoined
df_x11['_merge'].value_counts()
df_x11_both = df_x11[df_x11['_merge'] == 'both'].drop('_merge', axis=1) #both=17331
df_x11_left = df_x11[df_x11['_merge'] == 'left_only'].drop('_merge', axis=1)[df_n.columns] #left_only=569

#Check 
#df_x11[df_x11['council_name'] != df_x11['COUNCIL']]['COUNCIL'].value_counts() #FIXES 1.01-2
df_x11_both.shape

(17331, 27)

In [10]:
#1.2 LEFT JOIN on school name and council (for 1.1 unjoined)
df_x12 = df_x11_left.merge(df_ts16, how='left', indicator=True,\
                     left_on=['school_name', 'council_name'],\
                     right_on=['SCHOOL NAME', 'COUNCIL'])

df_x12.shape
#df_x12.to_csv('dataout/2a/df_x12_merged.csv')

#Split joined and unjoined
df_x12['_merge'].value_counts()
df_x12_both = df_x12[df_x12['_merge'] == 'both'].drop('_merge', axis=1) #both=389 => 489 => 567
df_x12_left = df_x12[df_x12['_merge'] == 'left_only'].drop('_merge', axis=1) #left_only=180 => 80 => 2

#df_x12_both.to_csv('dataout/2a/df_x12_both.csv')
#df_x12_left.to_csv('dataout/2a/df_x12_left.csv') #=> use for MANUAL Excel FIXES (above)

In [11]:
#1.3 Concatenate joined schools from 1.1 and 1.2, and unjoined from 1.2
df_x13 = pd.concat([df_x11_both, df_x12_both, df_x12_left])

#Check 
#df_x13.duplicated(subset='school_id', keep=False).sum() #duplicates=0
df_x13.shape
#df_x13.to_csv('dataout/2a/df_x13_merged.csv')

(17900, 27)

In [12]:
#1.4 Post-processing steps
df_x14a = df_x13.copy()

#Data sanity checks - NECTA vs. TAMISEMI (school info)
assert (df_x14a['council_name'] == df_x14a['COUNCIL']).all(), 'Found mismatch in council name' #expect no mismatches
assert (df_x14a['region_name'] == df_x14a['REGION']).all(), 'Found mismatch in region name' #expect no mismatches
#assert (df_x14a['school_id'] == df_x14a['NECTA EXAM CENTRE NO']).all(), 'Found mismatch in NECTA exam ID' #expect no mismatches

#Found duplicated "Registration Number" from "incorrectly" matched schools from 1.1
df_x14a['SCHOOL REG. NUMBER'].duplicated().sum() #result => 24 + 1 NaN
df_x14a_dup = df_x14a[df_x14a['SCHOOL REG. NUMBER'].duplicated(keep='last')] #first duplicates True
df_x14a_dup.to_csv('dataout/2a/df_x14a_duplicated_first.csv')
#Unmatch TAMISEMI data
df_x14a.loc[df_x14a['SCHOOL REG. NUMBER'].duplicated(keep='last'), df_ts16.columns] = pd.NA
df_x14a[df_x14a['SCHOOL REG. NUMBER'].notna()].duplicated().sum() #result => 0

#Use unique "NECTA exam ID" as index
df_x14a['school_id'].duplicated().sum() #check no duplicates
df_x14b = df_x14a.set_index('school_id')

#Drop unneeded columns (already used to cross-check)
df_x14c = df_x14b.drop(['REGION', 'COUNCIL', 'SCHOOL NAME', 'NECTA EXAM CENTRE NO'], axis=1)

#Save to CSV
df_x14c.shape
#df_x14c.to_csv('dataout/2a/df_x14_merged.csv')

(17900, 22)

### 2. Merge with TAMISEMI primary textbooks school data (PBR)
**ELI5 Summary:**
*Combine primary school textbooks data with merged NECTA+TAMISEMI data*

**Steps:**
1. Calculate PBR average feature per-school using `calc_pbr_average` < read in CSV
2. Light data cleaning
3. LEFT JOIN on indexes `'SCHOOL REG. NUMBER'`/`'Reg Number'`
4. Check MISSING `PBR_average` after merge
5. Remove unneeded columns > save CSV

**DATA observations:**
* Raw data contains **Pupil-to-Book Ratio (PBR)** for every standard-subject per-school
* There is no average PBR for all books so need to calculate using pupil and book counts per standard
* Some pupils and books have MISSING values so `fillna(0)` and skip in calculation

**Corner cases:**
* **3 schools** with NECTA data have zero books recorded making `'PBR_average'` = NaN **=> KEEP**

**Learnings:** (🧑🏻‍💻📚😎⚠️)
- 🧑🏻‍💻 Pandas `apply` function per DF school row is essential here
- ⚠️ MISSING data: `np.mean()` on an empty list causes a warning so need to catch

In [13]:
#Prepare TAMISEMI dataset #2 "Primary Textbooks" for joining
df_ts2 = pd.read_csv('../datain/Primary Textbooks and PBR, 2022.csv', index_col='Reg Number')
df_ts2.shape

(19261, 102)

In [14]:
#2.1 Calculate PBR average feature
df_ts21 = df_ts2.fillna(0)
df_ts21['PBR_average'] = df_ts21.apply(fn.calc_pbr_average, axis=1) #apply per DF school row
df_ts21.to_csv('dataout/2a/df_ts21_PBR_average.csv')

In [15]:
#2.2 Light data cleaning
#(1) Data type
df_ts21['PBR_average'].dtype #result: float64
#(2) Data values
df_ts21['PBR_average'].describe()
#(4) Remove unneeded columns
df_ts22 = df_ts21[['School Name', 'PBR_average']]
#(5) MISSING data
#df_ts22.info()
df_ts22['PBR_average'].isna().sum() #result: 27 => check after merge
#df_ts22[df_ts22['PBR_average'].isna()]

27

In [16]:
#2.3 LEFT JOIN on indexes 'SCHOOL REG. NUMBER'/'Reg Number'
df_x23 = df_x14c.merge(df_ts22, how='left', indicator=True,\
                       left_on=['SCHOOL REG. NUMBER'],\
                       right_index=True)

#Data sanity checks
df_x23['_merge'].value_counts()
df_x23.shape
df_x23['PBR_average'].describe()

count    17871.000000
mean         4.037403
std          4.178673
min          0.241935
25%          2.113054
50%          2.904063
75%          4.476140
max        124.076565
Name: PBR_average, dtype: float64

In [17]:
#2.4 Post processing

#Check MISSING data after merge
df_x23[df_x23['PBR_average'].isna()] #result: 29 rows = 26 no TAMISEMI match + 3 no books recorded

#Drop cross-check column
df_x24 = df_x23.drop(['School Name', '_merge'], axis=1)

#Save to CSV
df_x24.shape
#df_x24.to_csv('dataout/2a/df_x24_merged.csv')

(17900, 23)

### 3. Merge with TAMISEMI capitation grant school data (CG) 
**ELI5 Summary:**
*Combine primary school capitation grant data with merged NECTA+TAMISEMI data*

**Steps:**
1. Light data cleaning on raw data: strip whitespace < read in CSV
2. LEFT JOIN on indexes `'SCHOOL REG. NUMBER'`/`'Reg#'` > keep all
3. Calculate **Capitation Grant per student** (divide by all students)
4. Drop cross-check column > save CSV

**DATA observations:**
* Raw data contains per-school monthly (July 2021 - June 2022) grant amount in Tanzania Shillings (TZS) and total for financial year
* At first glance, most CG per student amounts are within a reasonable range
* Merge results:
    * **16255 schools** ARE getting grants (almost all Government)
    * **1645 schools** are NOT getting grants (almost all Non-Government) or not identifiable by registration ID

**Corner cases:**
* **1 school** (duplicated **ERROR**) has two CG entries with different amounts **=> DROP CG data** (as if not getting grants)
* **11 Non-Government** schools ARE getting grants
    * 5 of these have special/blind/deaf in their names
* **95 Government** schools are NOT getting grants

**Learnings:** (🧑🏻‍💻📚😎⚠️)
- 🧑🏻‍💻 Pandas string functions are useful here to cleanup whitespace
- 🧑🏻‍💻 Pandas vectorization at work!: one line division of column values to get 'School_CG_per_student' per school row

In [18]:
#Prepare TAMISEMI dataset #3 "School_CG" for joining
df_ts3 = pd.read_csv('../datain/School_CG_Primary_2021-2022.csv', header=2, index_col='Reg#')
df_ts3.shape

(16672, 22)

In [19]:
#3.1 Light data cleaning on raw data

#(2) Data values
#Strip whitespace
df_ts3.index = df_ts3.index.str.strip()
df_ts3['SCHOOL NAME (ts3)'] = df_ts3['SCHOOL NAME'].str.strip()
df_ts3['CG'] = df_ts3[' TOTAL '].str.strip().str.replace(',','').apply(lambda x : float(x))

#(1) Data type (after above)
df_ts3['CG'].dtype #result: float64

#(3) Duplicated rows
df_ts3[df_ts3.index.duplicated(keep=False)] #result: 2 rows with 'Reg# = 'EM.8020'
#DROP both duplicates
df_ts31 = df_ts3[~df_ts3.index.duplicated(keep=False)]

#(4) Remain with only required features
df_ts32 = df_ts31[['SCHOOL NAME (ts3)', 'CG']]

#(5) MISSING values
df_ts3[df_ts3['SCHOOL NAME (ts3)'].isna()] #result: 1 <= TOTAL row!
#DROP and recheck CG values
df_ts33 = df_ts32[df_ts32['SCHOOL NAME (ts3)'].notna()]

#Last check
#df_ts33['CG'].describe()
df_ts33.shape

(16669, 2)

In [20]:
#3.2 LEFT JOIN on indexes 'SCHOOL REG. NUMBER'/'Reg#'
df_x32 = df_x24.merge(df_ts33, how='left', indicator=True,\
                       left_on=['SCHOOL REG. NUMBER'],\
                       right_index=True)

df_x32['_merge'].value_counts() #both=16255 #left_only=1645
df_x32.shape

#df_x32[df_x32['_merge'] == 'both']['SCHOOL OWNERSHIP'].value_counts() #Receiving grants: Gov=16244, NonGov=11
#df_x32[df_x32['_merge'] == 'left_only']['SCHOOL OWNERSHIP'].value_counts() #NonGov=1524, Gov=95, NaN=26
#df_x32['SCHOOL OWNERSHIP'].isna().sum() #result: 26

(17900, 26)

In [21]:
#3.3 Calculate Capitation Grant per student (entire school)
#The power of vectorization!
df_x33b = df_x32.copy()
df_x33a = df_x32[df_x32['CG'].notna()] #avoid NaN values in 'CG_per_student'
df_x33b['CG_per_student'] = df_x33a['CG'] / df_x33a['TOTAL STUDENTS']
df_x33b[['CG', 'CG_per_student']].describe()

,CG,CG_per_student
count,1.625500e+04,16255.000000
mean,4.066465e+06,6409.352963
std,2.884566e+06,1716.295997
min,4.071692e+04,552.263985
25%,2.215315e+06,6021.859393
50%,3.418074e+06,6368.273899
75%,5.090749e+06,6706.914602
max,4.806562e+07,202332.886460


In [22]:
#3.4 Drop cross-check column
df_x34 = df_x33b.drop(['SCHOOL NAME (ts3)', '_merge'], axis=1)

#Save to CSV
df_x34.shape
#df_x34.to_csv('dataout/2a/df_x34_merged.csv')

(17900, 25)

### 4. Merge with TAMISEMI enrollment by age and sex school data
**ELI5 Summary:**
*Combine primary school enrollment ages data with merged NECTA+TAMISEMI data*

**Steps:**
1. Light data cleaning on raw data: strip whitespace < read in CSV
2. Calculate age features using `get_ages_stats`: SD, mean, median
3. LEFT JOIN on indexes `'SCHOOL REG. NUMBER'`/`'Reg.No.'`
4. Drop cross-check columns > save CSV

**DATA observations:**
* Raw data contains per-school enrollment of students by age and gender **=> very clean!**
* What's potentially useful?: decided that **mean, median, and standard deviation (SD) on all ages** makes sense
* Age stats will be approximate since we are assuming: Below 6 = 5, Above 13 = 14

**Learnings:** (🧑🏻‍💻📚😎⚠️)
- 🧑🏻‍💻 Use Pandas `to_list()` to turn tuple (from function) into seperate values (in columns)

In [23]:
#Prepare TAMISEMI dataset #4 "Enrollment" for joining
df_ts4 = pd.read_csv('../datain/Enrollment in Government and Non_Government Primary Schools by Age and Sex_2022.csv', index_col='Reg.No.')
df_ts4.shape

(19261, 28)

In [24]:
#4.1 Light data cleaning on raw data
df_ts41 = df_ts4.copy()

#(1) Data type (after above)
df_ts41.dtypes #result: all ages are int64

#(2) Data values
df_ts41.describe() #result: look reasonable

#(3) Duplicated rows
df_ts41[df_ts41.index.duplicated(keep=False)] #result: 0 rows

#(5) MISSING values
df_ts41.isna().sum().sum()

0

In [25]:
#4.2 Feature Extraction - ages SD, mean, median
df_ts41['student_ages'] = df_ts41.apply(fn.get_ages_stats, axis=1)
df_ts42 = pd.DataFrame(df_ts41['student_ages'].to_list(), columns = ['approx_ages_SD', 'approx_ages_mean', 'ages_median'], index=df_ts4.index)
df_ts42['School (ts4)'] = df_ts41['School'] #for cross-checking

#Check data values
df_ts42.isna().sum() #result: 5 for age stats (0 students) => will not be merged

approx_ages_SD      5
approx_ages_mean    5
ages_median         5
School (ts4)        0
dtype: int64

In [26]:
#4.3 LEFT JOIN on indexes 'SCHOOL REG. NUMBER'/'Reg.No.'
df_x43 = df_x34.merge(df_ts42, how='left', indicator=True,\
                      left_on=['SCHOOL REG. NUMBER'],\
                    right_index=True)

df_x43['_merge'].value_counts() #both=17871
#df_x43['approx_ages_SD'].isna().sum()
df_x43.shape

#df_x43[['approx_ages_SD', 'approx_ages_mean', 'ages_median']].describe()

(17900, 30)

In [27]:
#4.4 Drop cross-check column
df_x44 = df_x43.drop(['School (ts4)', '_merge'], axis=1)
df_x44.shape

#Save to CSV: final for Data Sourcing!
#df_x44.to_csv('dataout/2a/nation_merged.csv')

(17900, 28)

In [28]:
#SPOT-CHECK CODE - handy, keep around!
#df_x44.shape
df_x44.info()
#df_n4.describe(include='all')
#df_x24[df_x24['school_id'] == 'PS1104063'] #JITEGEMEE @Morogoro MC
#df_x24.loc['PS1104063'] #JITEGEMEE @Morogoro MC
#df_x14c.head()
#df_x13._is_copy

<class 'pandas.core.frame.DataFrame'>
Index: 17900 entries, PS0101114 to PS2001122
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   school_name               17900 non-null  string  
 1   num_sitters               17900 non-null  int64   
 2   average_300               17900 non-null  float64 
 3   grade                     17900 non-null  category
 4   region_name               17900 non-null  object  
 5   council_name              17900 non-null  object  
 6   num_sitters_girls         17900 non-null  int64   
 7   num_sitters_boys          17900 non-null  int64   
 8   ratio_sitters_girls_boys  17900 non-null  float64 
 9   pct_passed                17900 non-null  float64 
 10  approx_marks_SD_300       17900 non-null  float64 
 11  WARD                      17874 non-null  string  
 12  SCHOOL OWNERSHIP          17874 non-null  string  
 13  SCHOOL REG. NUMBER        17874 non-nul